## 手撕IoU
即两个框的交并比

In [ ]:
import numpy as np

# 给的 a[x1, y1, x2, y2] 分别是左上和右下的坐标
def ComputeIOU(a, b):
    # 求出相交四边形的左上角和右下角
    x1 = np.max(a[0], b[0])
    y1 = np.max(a[1], b[1])
    x2 = np.min(a[2], b[2])
    y2 = np.min(a[3], b[3])

    inter = np.max([x2-x1, 0]) * np.max([y2-y1, 0])
    union = (a[2]-a[0])*(a[3]-a[1]) + (b[2]-b[0])*(b[3]-b[1]) - inter

    return inter / union

## 手撕NMS 非极大值抑制
检测模型的输出结果会有很多重叠 需要从中选择最优的检测框  

### 算法流程
1. 根据置信度排序
2. 选取置信度最高的边界框添加到 最终的输出列表里面 从候选框删除
3. 计算候选框和其他框的IoU 删除IoU大于阈值的边界框

In [ ]:
import numpy as np

def nms(dets, score, threshold):
    if len(dets) == 0:
        return []
    
    boxes = np.array(dets)
    score = np.array(score)

    picked_boxes = []
    picked_score = []
    
    # 预先取出boxes的坐标 计算出框的面积
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    areas = (x2-x1+1)*(y2-y1+1)

    # 获得order 从小到大排列的元素序号
    order = np.argsort(score)

    while order.size > 0:
        index = order[-1]

        picked_boxes.append(boxes[index])
        picked_score.append(score[index])

        # 计算最大元素和其他框的IoU
        x1 = np.max(x1[index], x1[order[:-1]])
        y1 = np.max(y1[index], y1[order[:-1]])
        x2 = np.min(x2[index], x2[order[:-1]])
        y2 = np.min(y2[index], y2[order[:-1]])

        inter = np.max(0., x2-x1+1) * np.max(0., y2-y1+1)
        iou = inter / (areas[index] + areas[order[:-1]] - inter)
        order = order[np.where(iou < threshold)]  # 仅仅保留iou小于阈值的框

    return picked_boxes, picked_score

## 手撕匈牙利匹配 （相关性矩阵+状态压缩DP）